In [2]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString

In [3]:
# Define the file paths
bankline_folder = "D:/BUET/Thesis/ArcGIS/Waterbody and Bankline/Final_Bankline"
waterbody_folder = "D:/BUET/Thesis/ArcGIS/Waterbody and Bankline/Final_Waterbody"
cross_section_file = "D:/BUET/Thesis/ArcGIS/Cross_Section/cross_section_1km.shp"

In [6]:
# Load the cross section shapefile
cross_sections = gpd.read_file(cross_section_file)

# Initialize a dictionary to store the results
results = {}

# Loop through each year
for year in range(1988, 2025):
    # Load bankline and waterbody shapefiles for the given year
    bankline_file = f"{bankline_folder}/{year}.shp"
    waterbody_file = f"{waterbody_folder}/water_body_{year}.shp"
    bankline = gpd.read_file(bankline_file)
    waterbody = gpd.read_file(waterbody_file)

    # Initialize lists to store measurements for this year
    bank_to_bank_distances = []
    waterbody_lengths = []
    filtered_waterbody_lengths = []

    # Calculate distances for each cross section
    for _, cross_section in cross_sections.iterrows():
        cross_line = cross_section.geometry

        # Calculate bank to bank distance
        bankline_intersections = bankline[bankline.intersects(cross_line)]
        if len(bankline_intersections) == 2:
            bank_points = [bankline_intersections.iloc[i].geometry.intersection(cross_line) for i in range(2)]
            bank_to_bank_dist = bank_points[0].distance(bank_points[1])
        else:
            bank_to_bank_dist = None  # Set to None if the two banks are not detected

        # Calculate waterbody length along the cross section
        waterbody_intersections = waterbody[waterbody.intersects(cross_line)]
        total_waterbody_length = 0
        filtered_waterbody_length = 0

        for _, waterbody_part in waterbody_intersections.iterrows():
            intersected_length = waterbody_part.geometry.intersection(cross_line).length
            total_waterbody_length += intersected_length
            if intersected_length >= 80:
                filtered_waterbody_length += intersected_length

        # Store the results for this cross section
        bank_to_bank_distances.append(bank_to_bank_dist)
        waterbody_lengths.append(total_waterbody_length)
        filtered_waterbody_lengths.append(filtered_waterbody_length)

    # Add the data for the year to the results dictionary
    results[f"bank_to_bank_{year}"] = bank_to_bank_distances
    results[f"waterbody_length_{year}"] = waterbody_lengths
    results[f"filtered_waterbody_length_{year}"] = filtered_waterbody_lengths

# Create a DataFrame with cross section IDs and measurement data
output_df = pd.DataFrame(results)
output_df.index = cross_sections["ID"]  # Assuming 'id' column is cross section identifier

# Save to Excel
output_df.to_excel("bank_waterbody_distances.xlsx")